# Exercise 2: Transfer Learning

Attribution: Kolhatkar, Varada (2024) [DSCI 572](https://ubc-mds.github.io/DSCI_572_sup-learn-2/README.html) 

**Transfer learning** is like borrowing knowledge from one task to help with another: you take a model that has already learned patterns from a related task (e.g., classifying images in [Imagenet](https://www.image-net.org/)) and adapt it to your task (e.g., detecting specific types of fruits) with less effort and data. 

In this exercise, you will explore transfer learning by leveraging pre-trained image classification models. Specifically, you will:

- Use these models out of the box to classify your own images.
- Use them as feature extractors to obtain rich representations of your images, which you can then apply to your own tasks.

## Imports
<hr>

In [ ]:
from PIL import Image
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
from torchvision import datasets, models, transforms, utils
import glob
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import pandas as pd
import random
import torch
import torchvision
%matplotlib inline

plt.rcParams.update({'axes.grid': False})

## Getting Started with Kaggle Kernels
<hr>

We are going to run this notebook on the cloud using [Kaggle](https://www.kaggle.com). Kaggle offers 30 hours of free GPU usage per week which should be much more than enough for this lab. To get started, follow these steps:

1. Go to https://www.kaggle.com/kernels

2. Make an account if you don't have one, and verify your phone number (to get access to GPUs)
3. Select `+ New Notebook`
4. Go to `File -> Import Notebook`
5. Upload this notebook
6. On the right-hand side of your Kaggle notebook, make sure:
  
  - `Internet` is enabled.
  
  - In the `Accelerator` dropdown, choose `GPU` when you're ready to use it (you can turn it on/off as you need it).
    
7. Run the follow cells for preparation the model, labels and functions.

In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

The code in the following cell contains helper functions that will be used later. You don't need to fully understand this code to answer the questions in this notebook.

In [ ]:
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def classify_image(img, topn = 4):
    clf = models.vgg16(weights='VGG16_Weights.DEFAULT') # initialize the classifier with VGG16 weights
    preprocess = transforms.Compose([
                 transforms.Resize(299),
                 transforms.CenterCrop(299),
                 transforms.ToTensor(),
                 transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225]),])

    with open("imagenet_classes.txt", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    
    img_t = preprocess(img)
    batch_t = torch.unsqueeze(img_t, 0)
    clf.eval()
    output = clf(batch_t)
    _, indices = torch.sort(output, descending=True)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    d = {'Class': [classes[idx] for idx in indices[0][:topn]], 
         'Probability score': [np.round(probabilities[0, idx].item(),3) for idx in indices[0][:topn]]}
    df = pd.DataFrame(d, columns = ['Class','Probability score'])
    return df


# Attribution: [Code from PyTorch docs](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html?highlight=transfer%20learning)
IMAGE_SIZE = 200
BATCH_SIZE = 64

def read_data(data_dir, subdir):
    """
    Reads image data from the specified directory and applies transformations.
    """
    data_transforms = {
        "train": transforms.Compose(
            [
                transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),     
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),            
            ]
        ),
        "valid": transforms.Compose(
            [
                transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),                        
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),                        
            ]
        ),
    }

    image_datasets = {
        x: datasets.ImageFolder(os.path.join(data_dir, subdir[x]), data_transforms[x])
        for x in ["train", "valid"]
    }
    
    dataloaders = {}
    
    dataloaders["train"] = torch.utils.data.DataLoader(
            image_datasets["train"], batch_size=BATCH_SIZE, shuffle=True
        )
    
    dataloaders["valid"] = torch.utils.data.DataLoader(
            image_datasets["valid"], batch_size=BATCH_SIZE, shuffle=True
        )
    
    return image_datasets, dataloaders

def get_features(model, data_loader, seed=None):
    """Extract output of squeezenet model"""
    if seed:
        torch.manual_seed(seed)
    model.to(device)
    with torch.no_grad():  # turn off computational graph stuff
        Z_init = torch.empty((0, 1024)).to(device)  # Initialize empty tensors
        y_init = torch.empty((0)).to(device)
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            Z_init = torch.cat((Z_init, model(X)), dim=0)
            y_init = torch.cat((y_init, y))
    return Z_init.cpu().detach(), y_init.cpu().detach()

def show_predictions(pipe, Z_valid, y_valid, dataloader, class_names, num_images=20, seed=None):
    """Display images from the validation set and their predicted labels."""
    if seed:
        torch.manual_seed(seed)
    images_so_far = 0
    fig = plt.figure(figsize=(15, 25))  # Adjust the figure size for better visualization

    # Convert the features and labels to numpy arrays
    Z_valid = Z_valid.numpy()
    y_valid = y_valid.numpy()

    # Make predictions using the trained logistic regression model
    preds = pipe.predict(Z_valid)

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.cpu()
            for j in range(inputs.size()[0]):
                if images_so_far >= num_images:
                    return
                # print(f"Dataloader Labels: {labels[j]}: {class_names['valid'][labels[j]]}")
                ax = plt.subplot(num_images // 5, 5, images_so_far + 1)  # 5 images per row
                ax.axis('off')
                ax.set_title(f"Predicted: {class_names['train'][int(preds[images_so_far])]}"
                             f"\nTrue: {class_names['valid'][int(y_valid[images_so_far])]}")
                inp = inputs.data[j].numpy().transpose((1, 2, 0))
                mean = np.array([0.5, 0.5, 0.5])
                std = np.array([0.5, 0.5, 0.5])
                inp = std * inp + mean
                inp = np.clip(inp, 0, 1)
                ax.imshow(inp)
                #imshow(inputs.data[j])
                images_so_far += 1

def show_image_label_prob(image_path, true_label, num_images):
    """
    Displays a specified number of images from a given directory and prints their classification labels and probabilities.
    """
    images = glob.glob(image_path)
    selected_images = random.sample(images, num_images)
    plt.figure(figsize=(5, 5));
    for image in selected_images:
        img = Image.open(image)
        img.load()
        plt.imshow(img)
        plt.title(f'Actual Label: {true_label}')
        plt.show()
        df = classify_image(img)    
        print(df.to_string(index=False))
        print("--------------------------------------------------------------\n\n")

Once you've done all your work on Kaggle, you can download the notebook from Kaggle. That way any work you did on Kaggle won't be lost. 

## Exercise 1: Using pre-trained models out of the box
<hr>

First, we will use pre-trained Convolutional Neural Network (CNN) models out of the box for image classification. You can find a list of available pre-trained models [here](https://nnabla.readthedocs.io/en/v1.39.0/python/api/models/imagenet.html).

In this exercise, we'll use the `VGG16` model to classify cats and dogs using [this dataset](https://www.kaggle.com/datasets/tongpython/cat-and-dog). To get started with the dataset, follow the instructions below.

1. Click `+ Add Input` at the top right of the notebook.

2. Choose `Datasets`. In the search bar, type 'cat-and-dog'. Several datasets will appear. Locate and add the one with a size of 228MB.

Running the following cell will display the image, the model's predictions (`Class`), and the corresponding probabilities.

In [ ]:
# Set up data
IMAGE_DIR = {
    "Cat": "/kaggle/input/cat-and-dog/test_set/test_set/cats/*.*", 
    "Dog": "/kaggle/input/cat-and-dog/test_set/test_set/dogs/*.*"
}

# Display image and prediction labels with probability
for true_label, image_path in IMAGE_DIR.items():
    show_image_label_prob(image_path, true_label, num_images=4)

## Exercise 1.1 

<div class="alert alert-info">

**Discussion questions**

1. How well does the model distinguish between cats and dogs?
2. Do you notice any specific patterns or characteristics in the labels?
   
</div>

<div class="alert alert-warning">

Type your answer below.
    
</div>

_Type your answer here, replacing this text._

Hopefully, you observed reasonable performance on the cats and dogs dataset. Now, let's test the model on a slightly different dataset: [Fruit Classification Dataset](https://www.kaggle.com/datasets/karimabdulnabi/fruit-classification10-class). 

In order to use this dataset 

1. Click `+ Add Input` at the top right of the notebook.

2. Choose `Datasets`. In the search bar, type 'fruit-classification10-class'. Several datasets will appear. Locate and add the one with a size of 31MB.

The dataset includes images from the following 10 classes:
- Apple
- Banana
- Avocado
- Cherry
- Kiwi
- Mango
- Orange
- Pineapple
- Strawberries
- Watermelon

Running the following cell will display an image from this dataset, the model's predictions (`Class`), and the corresponding probabilities.

In [ ]:
# Set up data
IMAGE_DIR = {
    "Apple": "/kaggle/input/fruit-classification10-class/MY_data/train/Apple/*.jpeg", 
    "Banana": "/kaggle/input/fruit-classification10-class/MY_data/train/Banana/*.jpeg", 
    "Avocado": "/kaggle/input/fruit-classification10-class/MY_data/train/avocado/*.jpeg", 
    "Cherry": "/kaggle/input/fruit-classification10-class/MY_data/train/cherry/*.jpeg", 
    "Kiwi": "/kaggle/input/fruit-classification10-class/MY_data/train/kiwi/*.jpeg", 
    "Mango": "/kaggle/input/fruit-classification10-class/MY_data/train/mango/*.jpeg", 
    "Orange": "/kaggle/input/fruit-classification10-class/MY_data/train/orange/*.jpeg", 
    "Pineapple": "/kaggle/input/fruit-classification10-class/MY_data/train/pinenapple/*.jpeg", 
    "Strawberries": "/kaggle/input/fruit-classification10-class/MY_data/train/strawberries/*.jpeg", 
    "Watermelon": "/kaggle/input/fruit-classification10-class/MY_data/train/watermelon/*.jpeg", 
}

# Display image and prediction labels with probability
for true_label, image_path in IMAGE_DIR.items():
    show_image_label_prob(image_path, true_label, num_images=2)


## Exercise 1.2

<div class="alert alert-info">

**Discussion questions**

1. How well does the model distinguish between different types of fruits?
2. Did you notice any differences in the model's performance between the cats and dogs dataset and the fruits dataset? Briefly explain your answer. 

</div>

<div class="alert alert-warning">

Type your answer below.
    
</div>

_Type your answer here, replacing this text._

<br><br><br><br>

## Exercise 2: Using pre-trained models as feature extractors

<br>

Often, we want to train a model on our own datasets and have it predict classes specific to our data, rather than the 1000 classes from ImageNet. To achieve this, we can use pre-trained models as feature extractors. Specifically, we can leverage the rich representations learned by pre-trained models, use these representations as feature vectors, and train a new model on these feature vectors for our specific task.

In this exercise, you will use a pre-trained CNN model, `Densenet`, to extract features from images and train a logistic regression classifier to identify different types of fruits.

To get started, run the following cell to prepare the data.

In [ ]:
# Set up data
DATA_DIR = '/kaggle/input/fruit-classification10-class/MY_data/'
SUBDIR = {'train': 'train', 'valid': 'test'}
image_datasets, dataloaders = read_data(DATA_DIR, SUBDIR)
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "valid"]}
class_names = {"train": image_datasets["train"].classes,
               "valid": image_datasets["valid"].classes}

Let's look ar some sample images in the dataset. 

In [ ]:
# Plot samples
inputs, classes = next(iter(dataloaders["valid"]))
plt.figure(figsize=(10, 8)); 
plt.axis("off"); 
plt.title("Sample valid Images")
plt.imshow(np.transpose(utils.make_grid(inputs, padding=1, normalize=True),(1, 2, 0)));

Next, we'll extract feature vectors from the images above using the pre-trained model.

In [ ]:
# Download model and extract features
model = models.densenet121(weights="DenseNet121_Weights.IMAGENET1K_V1")
model.classifier = nn.Identity()  # remove that last "classification" layer
Z_train, y_train = get_features(
    model, dataloaders["train"], seed=42
)
Z_valid, y_valid = get_features(
    model, dataloaders["valid"], seed=42
)

We now have tabular data. Each example is represented with feature vectors extracted from the `Densenet` model.

In [ ]:
pd.DataFrame(Z_train).head(10)

We are now ready to train a logistic regression model using the extracted features above and `y_train`.

In [ ]:
# Train classification model
pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000))
pipe.fit(Z_train, y_train)
print("Training score: ", pipe.score(Z_train, y_train))
pipe.score(Z_valid, y_valid)
print("Validation score: ", pipe.score(Z_valid, y_valid))

Let's examine some of the predictions made by the model. 

In [ ]:
# Show predictions for 25 images in the validation set (5 rows of 5 images)
show_predictions(pipe, Z_valid, y_valid, dataloaders['valid'], class_names, num_images=25, seed=42)

## Exercise 2.1

<div class="alert alert-info">

**Discussion questions**

1. How well does the model distinguish between different types of fruits?
2. Is the performance better than the out-of-the-box performance? 

</div>

<div class="alert alert-warning">

Type your answer below.
    
</div>

_Type your answer here, replacing this text._

### Your Free Time (Optional)

**Your tasks**:

Choose any image dataset that interests you and train a model using it!

Feel free to discuss your ideas and progress with your teammates and the workshop team.

In [ ]:
DATA_DIR = "{_DATA_DIRECTORY_PATH_}"
SUBDIR = "{_SUB_DIRECTORY_PATH_}"
# Example - dataset `cat-breed-mardhik`
# DATA_DIR = '/kaggle/input/cat-breed/cat-breed/'
# SUBDIR = {'train': 'TRAIN', 'valid': 'TEST'}

# Set up data
image_datasets, dataloaders = read_data(DATA_DIR, SUBDIR)
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "valid"]}
class_names = {"train": image_datasets["train"].classes,
               "valid": image_datasets["valid"].classes}

# Download model and extract features
model = models.densenet121(weights="DenseNet121_Weights.IMAGENET1K_V1")
model.classifier = nn.Identity()  # remove that last "classification" layer
Z_train, y_train = get_features(
    model, dataloaders["train"], seed=42
)
Z_valid, y_valid = get_features(
    model, dataloaders["valid"], seed=42
)

# Train classification model
pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000))
pipe.fit(Z_train, y_train)
print("Training score: ", pipe.score(Z_train, y_train))
pipe.score(Z_valid, y_valid)
print("Validation score: ", pipe.score(Z_valid, y_valid))

In [ ]:
# Plot samples
show_predictions(pipe, Z_valid, y_valid, dataloaders['valid'], class_names, num_images=25, seed=42)

<!-- END QUESTION -->

<br><br>